# Data Processing Notebook
### This notebook goes through loading, cleaning and organising of the data ready for model training.

In [ ]:
# Using Python 3.11, Conda environment.
# Ensure using env.yml to create the environment (data_analysis_env). 
# Importing data analysis & visualisation librarys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Initial exploration of the data to gain understanding and inform cleaning/processing. 
###         ---- **CHECK THE DATA PATHS ARE CORRECT FOR YOUR LOCAL ENV** ----

In [ ]:
# Loading data into DataFrames.
# Please adjust paths accordingly from Drive link provided, do not download locally. 
spec_data_path = "../data/raw_data/order_specifications_csv.csv"
sched_data_path = "../data/raw_data/order_schedules_csv.csv"
spec_df = pd.read_csv(spec_data_path)
sched_df = pd.read_csv(sched_data_path)

# Strip the id column for it to be used as index and merging later on.
spec_df['id'] = spec_df['id'].str.strip()
sched_df['id'] = sched_df['id'].str.strip()

In [ ]:
print(f" There are {len(sched_df['product'].unique())} unique schedules in the data")
print(f" There are {sched_df['id'].nunique()} unique job numbers") 

In [ ]:
print(f"""The Schedule data looks like: 
      
{sched_df.head()}

...And the spec data looks like: 

{spec_df.head()}
""")

### Data shape manipulation: Spec

In [ ]:
# Drop unecessary columns
spec_df = spec_df.drop(columns=['type', 'spec_id'])

# Pivot the spec df so each job number is its own row and then ensure the index is a String object
wide_spec_df = spec_df.pivot(index='id', columns='question_no', values='answer')
wide_spec_df.index = wide_spec_df.index.astype(str)

### Data shape manipulation: Schedules

In [ ]:
# Drop unecessary columns
sched_df = sched_df.drop(columns=['x', 'y', 'z', 'a', 'b'])
sched_df = sched_df.dropna()

# Create a multi-hot encoding of the schedules so each possible schedule code has its own binary identifier
from sklearn.preprocessing import MultiLabelBinarizer
schedule_codes = sched_df.groupby('id')['x'].apply(list) # Create a df with a list of schedules for each job

mlb = MultiLabelBinarizer()
wide_sched_df = pd.DataFrame(mlb.fit_transform(schedule_codes), index=schedule_codes.index, columns=mlb.classes_)

### Merging both tables together on jobno (ID) & Aligning seperate DataFrames

There will be aligned_spec_df & aligned_sched_df dataframes that contain the spec data, aligned, shaped and pivoted. While also having a merged_df, merged dataframe of both datasets. 

In [ ]:
wide_spec_df = wide_spec_df.sort_index()
wide_sched_df = wide_sched_df.sort_index()

# Finding commmon ID's and creating aligned tables.
intersecting_jobs = wide_spec_df.index.intersection(wide_sched_df.index)

# Creating two seperate, aligned, tables for training.
aligned_spec_df = wide_spec_df[wide_spec_df.index.isin(intersecting_jobs)]
aligned_sched_df = wide_sched_df[wide_sched_df.index.isin(intersecting_jobs)]

# Strip whitespace from columns for cleaning in future
aligned_sched_df.columns = aligned_sched_df.columns.str.strip()

In [ ]:
aligned_spec_df.head(1)

In [ ]:
aligned_sched_df.head(1)

## Initial EDA & Cleaning

In [ ]:
# How common is each schedule? 
schedule_count = aligned_sched_df.sum().sort_values(ascending=True)
schedule_count.plot(kind='bar', figsize=(15,5))
plt.title('Schedule frequency')
plt.xticks(rotation=45)
plt.ylabel('Order count')
plt.xlabel('Schedule')
plt.tight_layout()
plt.show()

The above tells us that there are many orders that contain schedule codes unrelated to Insole manufacturing specifications. These are likely insole orders where there has also been another, seperate, product combined/added onto the order. Thus, we need to remove these other product codes from the data as they are irrelevant. For example, a BAPT3 is a Body Armour Pro-Term boot - not related to insoles. 
Two schedule codes appear really commonly, EZI-RIZE-MALE and EZI-RIZE-FEMALE, these are sepeart heel lift products, also not relevant to Insole manufacture. 

In [ ]:
# Delete all schedule columns that are unrelated to insole manufacture
insole_schedule_list = ['B36', 'B41', 'B43', 'B54A', 'B54B', 'B54C', 'B55A', 'B55B', 'B56']
aligned_sched_df = aligned_sched_df[insole_schedule_list]

In [ ]:
# Re-generate visualisation after cleaning. 
schedule_count = aligned_sched_df.sum().sort_values(ascending=True)
schedule_count.plot(kind='bar', figsize=(15,5))
plt.title('Schedule frequency')
plt.xticks(rotation=45)
plt.ylabel('Order count')
plt.xlabel('Schedule')
plt.tight_layout()
plt.show()

The above shows that there is high cardinality among the schedules. This will need to be considered in training. 

In [ ]:
# AFTER CLEANING, Creating a merged table for convenience
merged_df = aligned_spec_df.join(aligned_sched_df, how='inner')
merged_df.head(5)

In [ ]:
# Deleting free text columns in the merged dataframe for encoding in model training
free_text_cols = [18,20,22,24,27,30,52,62,64,66,68,71,74,96]
merged_df = merged_df.drop(columns=[18,20,22,24,27,30,52,62,64,66,68,71,74,96])
merged_df

## Exporting cleaned dataframes

In [ ]:
merged_df.to_csv('../data/processed/merged_data_base.csv', index=True)
aligned_sched_df.to_csv('../data/processed/clean_schedule_base.csv', index=True)
aligned_spec_df.to_csv('../data/processed/clean_specification_base.csv', index=True)